In [1]:
pip install kiwipiepy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install konlpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import numpy as np
import pandas as pd

C:\Users\cmhcm\AppData\Local\Temp\ipykernel_4064\1662815981.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## 14781개 뉴스 데이터 가져오기
- 풀본문 있는 데이터 입니다.

In [5]:
data_path = "C:/dev/openAI/result_df_14761.csv"
df = pd.read_csv(data_path, dtype=str)

df.head(3)

,news_id,news_date,news_keyword,news_feature,news_url,news_content,img_url
0,02100201.20240214165525001,2024-02-14,"나혼산,규현,바퀜,주문폭주,소요,배송,순수,원조,진공,용기,신세계,CJ,메가,플랫폼...","바퀜,규현,메이커빌,나혼산,신세계백화점,메가플랫폼,온오프라인,신세계,밀폐용기,보관력...",http://news.moneytoday.co.kr/view/mtview.php?n...,2024-02-14머니투데이 이동오 기자 순수 국내기술로 개발한 원조 진공용기신세계...,https://www.bigkinds.or.kr/resources/images/02...
1,02100201.20240214161101001,2024-02-14,"렌즈스토어,뷰티,렌즈,편집숍,윙크,울산,대구,오픈,뷰티,콘택트렌즈,플랫폼,윙크컴퍼니...","윙크컴퍼니,오프라인,뷰티렌즈,대구,울산,큐레이션,정문기,중구,남구,렌즈스토어",http://news.moneytoday.co.kr/view/mtview.php?n...,2024-02-14머니투데이 조한송 기자 뷰티 콘택트렌즈 플랫폼 윙크컴퍼니가 고객 ...,https://www.bigkinds.or.kr/resources/images/02...
2,01500601.20240214160335001,2024-02-14,"대표이사,대구불로탁주,최종국,대표,이사,생물,도전장,지역,막걸리,글로벌,시장,막걸리...","대구,대구불로탁주,불로막걸리,품평회,생막걸리,양조장,대한민국,한국막걸리협회,동구,생...",https://www.imaeil.com/page/view/2024021314385...,2024-02-14막걸리는 살아있는 생물입니다. 지역 전통 막걸리의 맛을 전국을 넘...,https://www.bigkinds.or.kr/resources/images/01...


# 중복제거 로직
- 형태소 분석기: Okt
- 벡터화: CountVectorizer
- 중복 기준: news_content 컬럼 데이터

In [6]:
from konlpy.tag import Okt

# 형태소 분석기 초기화
okt = Okt()

In [7]:
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer

# 전처리 함수 정의
def tokenizer(raw, pos=["Noun", "Alpha", "Number"], stopword=[]):
    return [
        word for word, tag in okt.pos(
            raw,
            norm=True,
            stem=True
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

## 전처리 및 벡터화

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vectorize = CountVectorizer(
    tokenizer=tokenizer,
    min_df=2
)

In [9]:
# 기사 본문에 나오는 단어의 빈도 계산
X = vectorize.fit_transform(df['news_content'])

c:\Users\cmhcm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [10]:
from sklearn.metrics.pairwise import cosine_similarity

# 코사인 유사도 계산
cos_sim = cosine_similarity(X)

# 코사인 유사도가 0.9 이상인 경우를 중복으로 판단
dup_idx = np.where(cos_sim > 0.9)

In [11]:
# 중복 기사 인덱스와 제목 출력 및 중복 기사 제거
to_drop = []

for i in range(len(dup_idx[0])):
    if dup_idx[0][i] != dup_idx[1][i]:  # 자기 자신과의 유사도는 제외

        # 먼저 게시된 기사를 남기고 싶으므로 인덱스가 더 작은 행을 제거
        to_remove = min(dup_idx[0][i], dup_idx[1][i])
        print("삭제되는 기사 제목: ", df.iloc[to_remove]['news_id'])
        print()

        to_drop.append(to_remove)

# 중복 기사 제거
df = df.drop(to_drop)
# 인덱스 재설정
df = df.reset_index(drop=True)

삭제되는 기사 제목:  02100101.20240213143503002

삭제되는 기사 제목:  02100101.20240213143503002

삭제되는 기사 제목:  02100501.20240212184757001

삭제되는 기사 제목:  02100501.20240212180734001

삭제되는 기사 제목:  02100501.20240212180734001

삭제되는 기사 제목:  07100501.20240212103856001

삭제되는 기사 제목:  07100501.20240212103856001

삭제되는 기사 제목:  02100501.20240212184757001

삭제되는 기사 제목:  07101201.20240207190030002

삭제되는 기사 제목:  02100801.20240207152738001

삭제되는 기사 제목:  07101201.20240207190030002

삭제되는 기사 제목:  02100801.20240207152738001

삭제되는 기사 제목:  07100501.20240207141448001

삭제되는 기사 제목:  02100851.20240207111614001

삭제되는 기사 제목:  02100851.20240207111614001

삭제되는 기사 제목:  02100851.20240207111614001

삭제되는 기사 제목:  02100851.20240207111502001

삭제되는 기사 제목:  02100851.20240207111614001

삭제되는 기사 제목:  02100851.20240207111502001

삭제되는 기사 제목:  07100501.20240207141448001

삭제되는 기사 제목:  02100311.20240207103903001

삭제되는 기사 제목:  02100311.20240207103903001

삭제되는 기사 제목:  01400351.20240206220218003

삭제되는 기사 제목:  01400351.20240206220218003

삭제되는 기사 제목:  015

### 4,353개의 중복기사 제거

In [12]:
# 중복제거 확인
df.tail()

,news_id,news_date,news_keyword,news_feature,news_url,news_content,img_url
10424,01100901.20240215114426001,2024-02-15,"K-패션,세계,전통,12월,삼성패션연구소,2023년,마무리,한국,패션,시장,이슈,K...","서울,한국,블랙핑크,펀딩,영국,온라인,전통문화,굿즈,오우르,디자,현대,문체부,공진원...",https://www.joongang.co.kr/article/25228843,2024-02-15작년 12월 삼성패션연구소는 2023년 한 해를 마무리하며 202...,https://www.bigkinds.or.kr/resources/images/01...
10425,01400201.20240215112047001,2024-02-15,"충남,관광,얼굴,가디,탄생,도조,참매,형상,무령왕릉,석수,가디,캐릭터,충남,관광,사...","가디,충남,충남문화관광재단,참매,백제,도조,굿즈,무령왕릉,충청도,천연기념물,환경부,...",https://www.daejonilbo.com/news/articleView.ht...,2024-02-15충남 얼굴이 될 워디 가디가 태어났다. 충남 관광 캐릭터로 활동할...,https://www.bigkinds.or.kr/resources/images/01...
10426,02100201.20240215111347001,2024-02-15,"피앳,유즈,제인마치,메종,성수동,오픈,프랑스,감성,컬래버레이션,팝업,비비안,피앳유즈...","컬래버레이션,피앳유즈,비비안,메종,라운지웨어,프랑스,성수동,에코백,제인마치,팝업스토...",http://news.moneytoday.co.kr/view/mtview.php?n...,2024-02-15머니투데이 김건우 기자 비비안의 신규 브랜드 피앳유즈가 제인마치 ...,https://www.bigkinds.or.kr/resources/images/02...
10427,02100501.20240215102941001,2024-02-15,"창업,고양시,중심지,혁신,창업,창업,지원,활성,계획,수립,혁신,창업,기업,발굴,육성...","고양,고양시,스타트업,창업자,고양벤처펀드,특례시,고양특례시,중심지",http://www.fnnews.com/news/202402151023453144,2024-02-152024년 창업지원 활성화 계획 수립 혁신 창업기업 발굴육성 박차...,https://www.bigkinds.or.kr/resources/images/02...
10428,02100201.20240215070053001,2024-02-15,"웹툰,콘티,AI,LLM,교수,창업,도전,대표,송민,오노마,AI,기사,스타트업,기업정...","웹툰,오노마ai,스타트업,송민,ai,투툰,러시아,llm,팝업스토어",http://news.moneytoday.co.kr/view/mtview.php?n...,2024-02-15머니투데이 남미래 기자 스타트UP스토리 송민 오노마AI 대표이 기...,https://www.bigkinds.or.kr/resources/images/02...


In [13]:
df['news_feature']

0        바퀜,규현,메이커빌,나혼산,신세계백화점,메가플랫폼,온오프라인,신세계,밀폐용기,보관력...
1               윙크컴퍼니,오프라인,뷰티렌즈,대구,울산,큐레이션,정문기,중구,남구,렌즈스토어
2        대구,대구불로탁주,불로막걸리,품평회,생막걸리,양조장,대한민국,한국막걸리협회,동구,생...
3        엄브로,강남점,영국,혼네,신세계백화점,주년,론칭,굿즈,엄브,일상복,선현우,신세계,팝...
4        영국,엄브로,엄브,신세계백화점,주년,론칭,강남점,신세계,축구팀,일상복,오랫동안,선현...
                               ...                        
10424    서울,한국,블랙핑크,펀딩,영국,온라인,전통문화,굿즈,오우르,디자,현대,문체부,공진원...
10425    가디,충남,충남문화관광재단,참매,백제,도조,굿즈,무령왕릉,충청도,천연기념물,환경부,...
10426    컬래버레이션,피앳유즈,비비안,메종,라운지웨어,프랑스,성수동,에코백,제인마치,팝업스토...
10427                 고양,고양시,스타트업,창업자,고양벤처펀드,특례시,고양특례시,중심지
10428                 웹툰,오노마ai,스타트업,송민,ai,투툰,러시아,llm,팝업스토어
Name: news_feature, Length: 10429, dtype: object

# 모델링

In [14]:
# 카피 데이터프레임 만들기
df_copy = df.copy()

## 학습에 사용될 'nesw_feature' 컬럼 전처리
- 고유명사 추출 후 제거

In [15]:
# "원본데이터"를 대상으로 "kiwi"를 사용해서 형태소 분석하기
from itertools import product
from kiwipiepy import Kiwi
kiwi = Kiwi()

In [16]:
# 형태소 분석 및 고유명사 추출 함수 정의
def analyze_and_extract_NNP(text):
    result = kiwi.analyze(text)
    NNP_words = [word for word, tag, _, _ in result[0][0] if tag == 'NNP']
    return NNP_words

In [17]:
# 쉼표 제거 후 형태소 분석 및 저장
df_copy['news_feature'] = df_copy['news_feature'].str.replace(',', ' ')  # 쉼표 제거
df_copy['NNP'] = df_copy['news_feature'].apply(analyze_and_extract_NNP)

In [18]:
# news_feature 열 데이터를 단어로 분할하여 새로운 열 생성
df_copy['news_feature_words'] = df_copy['news_feature'].apply(lambda x: x.split())

In [19]:
# 'news_feature' 열 데이터와 'NNP' 열 데이터 비교하여 겹칠 경우 삭제
for index, row in df_copy.iterrows():
    nnp_list = row['NNP']
    news_feature = row['news_feature']
    cleaned_news_feature = ' '.join(word for word in news_feature.split() if word not in nnp_list)
    df_copy.at[index, 'news_feature_cleaned'] = cleaned_news_feature



In [20]:
df_copy.drop(['NNP', 'news_keyword'], axis=1, inplace=True)

In [21]:
df_copy['news_feature_cleaned']

0                       메가플랫폼 온오프라인 밀폐용기 보관력 온라인 짭퀜 프코크 cj
1                                윙크컴퍼니 오프라인 뷰티렌즈 정문기 렌즈스토어
2             대구불로탁주 불로막걸리 품평회 생막걸리 양조장 한국막걸리협회 동구 생효모 경쟁력
3        강남점 주년 론칭 굿즈 엄브 일상복 선현우 팝업스토어 선착순 듀오 런드리 20만 포...
4        주년 론칭 강남점 축구팀 일상복 오랫동안 선현우 팝업스토어 블록코어 매력적 믹스매치...
                               ...                        
10424                    블랙핑크 펀딩 온라인 전통문화 문체부 공진원 한복상점 와디즈
10425    충남문화관광재단 천연기념물 환경부 서흥식 스토리텔링 충남방문 캠핑페스티벌 투어패스 ...
10426    컬래버레이션 피앳유즈 라운지웨어 에코백 제인마치 팝업스토어 제품들 피앳 스타필드수원...
10427                            스타트업 창업자 고양벤처펀드 고양특례시 중심지
10428                        오노마ai 스타트업 송민 ai 투툰 llm 팝업스토어
Name: news_feature_cleaned, Length: 10429, dtype: object

## news_feature_cleaned 컬럼 데이터 벡터화
- 벡터화 방식: FastText
- 유사도 계산: 맨허튼

In [22]:
from gensim.models.fasttext import load_facebook_model

# FastText 모델 로드
ko_model = load_facebook_model("C:/Users/cmhcm/OneDrive/바탕 화면/cc.ko.300.bin")

In [23]:
from sklearn.metrics.pairwise import cosine_similarity

# 데이터 프레임에서 텍스트 데이터를 가져옵니다.
texts = df_copy['news_feature_cleaned'].values

# 텍스트를 단어 리스트로 변환합니다.
word_lists = [text.split() for text in texts]

In [24]:
# FastText 모델에 데이터를 추가 학습시킵니다.
ko_model.build_vocab(word_lists, update=True)
ko_model.train(word_lists, total_examples=len(word_lists), epochs=ko_model.epochs)

(45612, 145449)

In [25]:
from sklearn.metrics.pairwise import pairwise_distances

# FastText 벡터화를 수행합니다.
vectors = []
for text in texts:
    words = [word for word in text.split() if word in ko_model.wv]
    if words:
        vector = np.mean([ko_model.wv[word] for word in words], axis=0)
    else:
        vector = np.zeros(ko_model.vector_size)  # 모델의 벡터 크기에 맞는 제로 벡터를 사용
    vectors.append(vector)

vectors = np.array(vectors)  # 벡터 리스트를 numpy array로 변환

# 맨해튼 거리를 사용하여 유사도를 계산합니다.
manhattan_similarities = pairwise_distances(vectors, metric='manhattan')


In [26]:
# 맨허튼 유사도를 출력합니다.
print(manhattan_similarities)

[[0.         5.36078887 8.90072533 ... 6.33822706 8.64346174 7.75671266]
 [5.36078887 0.         8.13443676 ... 5.58279249 7.96037118 7.12334074]
 [8.90072533 8.13443676 0.         ... 8.38270908 9.83598849 9.51423597]
 ...
 [6.33822706 5.58279249 8.38270908 ... 0.         8.24575018 6.89008019]
 [8.64346174 7.96037118 9.83598849 ... 8.24575018 0.         9.13494545]
 [7.75671266 7.12334074 9.51423597 ... 6.89008019 9.13494545 0.        ]]


In [32]:
# 각 행마다 유사도가 낮은 상위 6개의 인덱스를 추출합니다.
top_6_indices = np.argsort(manhattan_similarities, axis=1)[:, :6]

# 첫 번째 항목 (즉, 자기 자신)을 제외합니다.
top_5_indices = top_6_indices[:, 1:]

# 해당 인덱스를 사용하여 news_ID를 찾습니다.
top_5_news_IDs = df_copy.loc[top_5_indices.flatten(), 'news_id'].values.reshape(-1, 5)

# 'Top_5'라는 새로운 컬럼에 news_ID를 저장합니다.
df_copy['Top_5'] = top_5_news_IDs.tolist()


In [33]:
# 자기 자신과의 유사도를 제외하고 각 행에서 가장 낮은 유사도를 찾습니다.
np.fill_diagonal(manhattan_similarities, np.inf)
best_similarities = np.min(manhattan_similarities, axis=1)

# 'Best_similarities'라는 새로운 컬럼에 최소 유사도를 저장합니다.
df_copy['Best_similarities'] = best_similarities


In [34]:
df_copy.head()

,news_id,news_date,news_feature,news_url,news_content,img_url,news_feature_words,news_feature_cleaned,Top_5,Best_similarities
0,02100201.20240214165525001,2024-02-14,바퀜 규현 메이커빌 나혼산 신세계백화점 메가플랫폼 온오프라인 신세계 밀폐용기 보관력...,http://news.moneytoday.co.kr/view/mtview.php?n...,2024-02-14머니투데이 이동오 기자 순수 국내기술로 개발한 원조 진공용기신세계...,https://www.bigkinds.or.kr/resources/images/02...,"[바퀜, 규현, 메이커빌, 나혼산, 신세계백화점, 메가플랫폼, 온오프라인, 신세계,...",메가플랫폼 온오프라인 밀폐용기 보관력 온라인 짭퀜 프코크 cj,"[02100801.20200828091925001, 07100501.20220906...",4.618928
1,02100201.20240214161101001,2024-02-14,윙크컴퍼니 오프라인 뷰티렌즈 대구 울산 큐레이션 정문기 중구 남구 렌즈스토어,http://news.moneytoday.co.kr/view/mtview.php?n...,2024-02-14머니투데이 조한송 기자 뷰티 콘택트렌즈 플랫폼 윙크컴퍼니가 고객 ...,https://www.bigkinds.or.kr/resources/images/02...,"[윙크컴퍼니, 오프라인, 뷰티렌즈, 대구, 울산, 큐레이션, 정문기, 중구, 남구,...",윙크컴퍼니 오프라인 뷰티렌즈 정문기 렌즈스토어,"[02100311.20230303134749001, 02100201.20220408...",3.016556
2,01500601.20240214160335001,2024-02-14,대구 대구불로탁주 불로막걸리 품평회 생막걸리 양조장 대한민국 한국막걸리협회 동구 생...,https://www.imaeil.com/page/view/2024021314385...,2024-02-14막걸리는 살아있는 생물입니다. 지역 전통 막걸리의 맛을 전국을 넘...,https://www.bigkinds.or.kr/resources/images/01...,"[대구, 대구불로탁주, 불로막걸리, 품평회, 생막걸리, 양조장, 대한민국, 한국막걸...",대구불로탁주 불로막걸리 품평회 생막걸리 양조장 한국막걸리협회 동구 생효모 경쟁력,"[01100401.20230727030456001, 01500501.20220329...",6.277194
3,07100501.20240214144618001,2024-02-14,엄브로 강남점 영국 혼네 신세계백화점 주년 론칭 굿즈 엄브 일상복 선현우 신세계 팝...,http://www.etnews.com/20240214000260,2024-02-14신세계백화점은 영국 스포츠 브랜드 엄브로 론칭 100주년을 맞아 ...,https://www.bigkinds.or.kr/resources/images/07...,"[엄브로, 강남점, 영국, 혼네, 신세계백화점, 주년, 론칭, 굿즈, 엄브, 일상복...",강남점 주년 론칭 굿즈 엄브 일상복 선현우 팝업스토어 선착순 듀오 런드리 20만 포...,"[02100501.20240214103102002, 01500701.20210927...",5.459882
4,02100501.20240214103102002,2024-02-14,영국 엄브로 엄브 신세계백화점 주년 론칭 강남점 신세계 축구팀 일상복 오랫동안 선현...,http://www.fnnews.com/news/202402141025245334,2024-02-14파이낸셜뉴스 신세계백화점은 영국 스포츠 브랜드 엄브로의 론칭 10...,https://www.bigkinds.or.kr/resources/images/02...,"[영국, 엄브로, 엄브, 신세계백화점, 주년, 론칭, 강남점, 신세계, 축구팀, 일...",주년 론칭 강남점 축구팀 일상복 오랫동안 선현우 팝업스토어 블록코어 매력적 믹스매치...,"[07100501.20231120164122002, 01100401.20240111...",4.907362


In [30]:
df.loc[df['news_id'] == '02100201.20210118095209001']

,news_id,news_date,news_keyword,news_feature,news_url,news_content,img_url
9235,02100201.20210118095209001,2021-01-18,"로이비,LOiViE,신세계백화점,강남점,팝업,스토어,오픈,파미에스트리트,신세계백화점...","로이비,오프라인,강남점,팝업스토어,신세계인터내셔날,신세계백화점,본고장,론칭,파미에스...",http://news.moneytoday.co.kr/view/mtview.php?n...,2021-01-18머니투데이 오정은 기자 이달 15일 신세계백화점 강남점 파미에스트...,https://www.bigkinds.or.kr/resources/images/02...


# 최종 평균 유사도: 5.131376852764612

In [35]:
# 'Best_similarities' 컬럼의 평균 유사도를 계산합니다.
average_similarity = df_copy['Best_similarities'].mean()

print("평균 유사도: ", average_similarity)

평균 유사도:  5.131376852764612
